In [1]:
# Importing necessary libraries from TensorFlow
pip install tensorflow
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
# Creating an ImageDataGenerator for data augmentation
data_generator = ImageDataGenerator(
    # Configuring image augmentation parameters
    rescale=1.0/255.0,  # Rescaling the image
    zoom_range=0.4,  # Randomly zooming the image
    horizontal_flip=True,  # Randomly flipping the image horizontally
    vertical_flip=True,  # Randomly flipping the image vertically
    rotation_range=50,  # Randomly rotating the image
    width_shift_range=0.3,  # Randomly shifting the width of the image
    height_shift_range=0.3,  # Randomly shifting the height of the image
    shear_range=0.4,  # Shear Intensity (Shear angle in counter-clockwise direction in degrees)
    fill_mode='nearest'  # Filling in the area outside the input image
)
# Loading the training data from the 'PetImages' directory
train = data_generator.flow_from_directory('PetImages/', class_mode='binary', batch_size=50, target_size=(200, 200))
# Using flow_from_directory() to load images in batches from the directory
# This approach helps avoid loading all images into RAM at once, making it memory efficient.

In [ ]:
# Checking the data generator
train

In [ ]:
# Loading the test data from the 'PetImagesTest' directory
test = data_generator.flow_from_directory('PetImagesTest/', class_mode='binary', batch_size=50, target_size=(200, 200))

In [ ]:
# Creating a Sequential model
model = Sequential()
# Adding convolutional and pooling layers
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(200, 200, 3)))
model.add(MaxPool2D((2, 2)))
# Flattening the input for the dense layer
model.add(Flatten())
# Adding a dense layer with sigmoid activation for binary classification
model.add(Dense(1, activation='sigmoid'))

In [ ]:
# Displaying the model summary
model.summary()

In [ ]:
# Compiling the model
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Checking for bad (corrupted) images in the test directory and handling them
import os # this function ssearch in folder and find bad(destroy) images and if it doesn't prepare --> introduce them 
from PIL import Image
# Defining the folder path and empty list to store extensions
folder_path = 'PetImagesTest'
extensions = []
# Iterating through the directories and files to find and handle bad images
for fldr in os.listdir(folder_path):
    sub_folder_path = os.path.join(folder_path, fldr)
    for filee in os.listdir(sub_folder_path):
        file_path = os.path.join(sub_folder_path, filee)
        print('** Path: {}  **'.format(file_path), end="\r", flush=True)
        im = Image.open(file_path)
        rgb_im = im.convert('RGB')
        # Checking and appending unique extensions
        if filee.split('.')[1] not in extensions:
            extensions.append(filee.split('.')[1])

In [ ]:
# Training the model using data generator
history = model.fit_generator(train, steps_per_epoch=len(train), validation_data=test, validation_steps=len(test), epochs=30) # steps_per_epoch=len(train) = because we don't know how many image we have --> set this to finished each epoch when process as count as length of train data

In [ ]:
# Plotting the training and validation accuracy
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'], color='green')
plt.plot(history.history['val_accuracy'], color='blue')

In [ ]:
# Saving the trained model
model.save('catvsdog.h5')